# Generate and save ECODE protocols and features 

This notebook generates E-CODE protocols, responses, and feature files to be used for optimizations.

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import MEAutility as mu
from pprint import pprint
import time
import numpy as np
import neuroplotlib as nplt

import multimodalfitting as mf

import sys
import shutil

from pathlib import Path
import os

%matplotlib notebook

## 0) Define model 

In [ ]:
# general
model_name = "hay"
probe_type = "planar" # linear 
cell_models_folder = Path("..") / "cell_models"
model_folder = cell_models_folder / model_name

fitting_folder = model_folder / "fitting" 
responses_folder = fitting_folder / "responses"
extra_folder = fitting_folder / "extracellular"

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()

## Define standard deviation for features

In [ ]:
std_from_mean = 0.05

In [ ]:
# define rheobase min step and step increment
rheobase_compute_params = {
    "hay": {"step_min": 0.3, 
            "step_increment": 0.02},
    "hay_ais": {"step_min": 0.15, 
                "step_increment": 0.005},
    "hay_ais_hillock": {"step_min": 0.15, 
                        "step_increment": 0.005},
}

## 1) Generate and run ecode protocols

We first define the cell model, probe, and simulator:

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name, release=True)
cell_unfrozen = mf.create_ground_truth_model(model_name=model_name)

probe = mf.define_electrode(probe_type=probe_type)

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe,
                                     mechs_folders=model_folder)

param_names = sorted([param.name for param in cell_unfrozen.params.values() if not param.frozen])
# sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

params_release = {}
for param in cell.params_by_names(param_names):
    params_release[param.name] = param.value
pprint(params_release)

In [ ]:
from multimodalfitting.ecode import compute_rheobase_for_model, generate_ecode_protocols, run_ecode_protocols,\
    save_intracellular_responses, save_extracellular_template

Then we find the rheobase for the model:

In [ ]:
rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, 
                                                                      **rheobase_compute_params[model_name])

Given the rheobase, we can compute the correct amplitudes of the ECODE stimuli. The `IDrest_300` sweep (300% rheobase) will be used to compute extracellular action potentials, as it contains the highest number of spikes.

In [ ]:
ecode_protocols = generate_ecode_protocols(rheobase_current=rheobase, record_extra=True,
                                           protocols_with_lfp="IDrest_300")

In [ ]:
print(f"ECODE protocols: {list(ecode_protocols.keys())}")

We then simulate the responses, that we are going to be used to extract features (this might take a while...):

In [ ]:
responses_dict = run_ecode_protocols(protocols=ecode_protocols, cell=cell, sim=sim, 
                                     resample_rate_khz=40)

In [ ]:
save_intracellular_responses(responses_dict=responses_dict, output_folder=responses_folder)

In [ ]:
eap, locations = save_extracellular_template(responses=responses_dict["IDrest"], 
                                             protocols=ecode_protocols, protocol_name="IDrest",
                                             probe=probe, output_folder=extra_folder, sweep_id=10, 
                                             **extra_kwargs)

In [ ]:
mf.plot_responses(responses_dict["IDrest"])

## 2) Generate feature and protocols with BluePyEfe

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

In [ ]:
from multimodalfitting.efeatures_extraction import build_model_metadata, model_csv_reader, get_ecode_targets

In [ ]:
efeatures_output_directory = fitting_folder / "efeatures"

In [ ]:
files_metadata = build_model_metadata(cell_id=model_name, ephys_dir=responses_folder)
pprint(files_metadata[model_name])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=model_csv_reader
)

### Define targets

Targets describe the features that we want to extract from the different protocols. Some protocols require also to set the timing to extract some features on. The `get_ecode_target` function is a helper function that returns the targets for the ECODE stimuli. The target dictionary can be adjusted/modified for other kind of protocols.

In [ ]:
# get timing information from the ECODE protocols
ecode_timings = mf.efeatures_extraction.ecodes_model_timings
pprint(ecode_timings)

In [ ]:
# define target features for different protocols
targets = get_ecode_targets(ecode_timings)
pprint(targets)

In [ ]:
targets = convert_legacy_targets(targets)

In [ ]:
t_start = time.time()
extract_efeatures_at_targets(cells, targets, efel_settings={'interp_step': 0.1})
t_stop = time.time()
print(f"Elapsed time {t_stop - t_start}")

In [ ]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres']
)

In [ ]:
print(f"Rheobase: {cells[0].rheobase}")

In [ ]:
protocols = group_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=efeatures_output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

## 3) Convert to BPO format and append extra features

In [ ]:
from multimodalfitting.efeatures_extraction import compute_extra_features, convert_to_bpo_format,\
    append_extrafeatures_to_json

We can now define the protocols we want to use for optimization:

In [ ]:
protocols_of_interest = ["IDrest_150", "IDrest_250", "IDrest_300", "IV_-100", "IV_-20", "APWaveform_260"]

in_protocol_path = efeatures_output_directory / "protocols.json"
in_efeatures_path = efeatures_output_directory / "features.json"

### a) All channels

In [ ]:
out_protocol_path_all = efeatures_output_directory / "protocols_BPO_all.json"
out_efeatures_path_all = efeatures_output_directory / "features_BPO_all.json"

In [ ]:
# append MEA.LFP features
eap = np.load(fitting_folder / "extracellular" / "template.npy")
fs = np.load(fitting_folder / "extracellular" / "fs.npy")
locations = np.load(fitting_folder / "extracellular" / "locations.npy")

In [ ]:
eap_norm = eap / np.max(np.abs(eap), 1, keepdims=True)
# vscale = 2
ax = mu.plot_mea_recording(eap, probe)

In [ ]:
extra_features = compute_extra_features(eap, fs, upsample=10)

In [ ]:
pprint(extra_features["peak_to_valley"])

In [ ]:
protocols_dict_all, efeatures_dict_all = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                               out_protocol_path_all, 
                                                               out_efeatures_path_all, 
                                                               protocols_of_interest=protocols_of_interest, 
                                                               std_from_mean=std_from_mean)

In [ ]:
channel_ids_all = None
single_channel_features = False

In [ ]:
extrafeatures_dict_all = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                      efeatures_dict=efeatures_dict_all,
                                                      efeatures_path=out_efeatures_path_all, 
                                                      channel_ids=channel_ids_all,
                                                      single_channel_features=single_channel_features, 
                                                      std_from_mean=None)

In [ ]:
# check MEA features
pprint(extrafeatures_dict_all["extra"]["IDrest_300"]["MEA"])

In [ ]:
probe.info

In [ ]:
probe_info = probe.info
probe_info["center"] = False
probe_info["pos"] = list([list(pos) for pos in probe.positions])
# del probe_info["pitch"], probe_info["dim"]
pprint(probe_info)

In [ ]:
json.dump(probe_info, (efeatures_output_directory / "probe_BPO.json").open("w"))
np.save(efeatures_output_directory / "template_BPO.npy", eap)

In [ ]:
# plot one extra features
f = mf.plot_feature_map_w_colorbar(extra_features["peak_to_valley"], probe, 
                                   feature_name="pos_image", label="time (s)")

### b) Select single channels

In [ ]:
out_protocol_path_single = efeatures_output_directory / "protocols_BPO_single.json"
out_efeatures_path_single = efeatures_output_directory / "features_BPO_single.json"

In [ ]:
protocols_dict_single, efeatures_dict_single = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                     out_protocol_path_single, 
                                                                     out_efeatures_path_single, 
                                                                     protocols_of_interest=protocols_of_interest, 
                                                                     std_from_mean=std_from_mean)

In [ ]:
channel_ids_single = mf.select_single_channels(cell, sim, probe)

In [ ]:
channel_ids_single = [36, 29, 24, 64, 43, 44, 2]

In [ ]:
channel_ids_single

In [ ]:
single_channel_features = True

In [ ]:
extrafeatures_dict_single = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                         efeatures_dict=efeatures_dict_single,
                                                         efeatures_path=out_efeatures_path_single, 
                                                         channel_ids=channel_ids_single,
                                                         single_channel_features=single_channel_features, 
                                                         std_from_mean=std_from_mean)

In [ ]:
pprint(extrafeatures_dict_single["extra"]["IDrest_300"]["MEA"])

### c) Select channel sections

In [ ]:
out_protocol_path_sections = efeatures_output_directory / "protocols_BPO_sections.json"
out_efeatures_path_sections = efeatures_output_directory / "features_BPO_sections.json"

In [ ]:
protocols_dict_sections, efeatures_dict_sections = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                         out_protocol_path_sections, 
                                                                         out_efeatures_path_sections, 
                                                                         protocols_of_interest=protocols_of_interest, 
                                                                         std_from_mean=std_from_mean)

In [ ]:
channel_ids_sections = mf.select_mea_sections(cell, sim, probe)

In [ ]:
channel_ids_sections = [[34, 35, 36, 37, 54, 55, 56, 57],
                        [25, 26, 27, 28, 45, 46, 47, 48],
                        [2, 22, 23, 24, 42, 43, 44, 64]]

In [ ]:
channel_ids_sections

In [ ]:
single_channel_features = False
std_from_mean = None

In [ ]:
extrafeatures_dict_sections = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                           efeatures_dict=efeatures_dict_sections,
                                                           efeatures_path=out_efeatures_path_sections, 
                                                           channel_ids=channel_ids_sections,
                                                           single_channel_features=single_channel_features, 
                                                           std_from_mean=std_from_mean)

In [ ]:
pprint(extrafeatures_dict_sections["extra"]["IDrest_300"])